In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline

from __future__ import division

import brightside as bs
import numpy as np
import ntcir
import ntcir.IO as IO
import os
import os.path as osp
import re
import itertools
import pickle
import random

import errno

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import collections
from easydict import EasyDict as edict
import pandas as pd

from skimage import io
import shutil
from collections import defaultdict

In [3]:
users = IO.load_annotations(ntcir.filepaths)
sorted_users = ntcir.utils.sort(users)
categories = IO.load_categories(ntcir.filepaths)

days = defaultdict(lambda: defaultdict(ntcir.Day))
for user in sorted_users:
    for day in user.days:
        days[user.id_][day.date] = day

In [14]:
def makedirs(dir_path):
    try:
        os.makedirs(dir_path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(dir_path):
            pass
        else:
            raise
            
def num_digits(number):
    return int(np.floor(np.log10(np.abs(number))) + 1)

def ext(path):
    return os.path.splitext(path)[-1]

In [14]:
def link_images(num_categories, split_dir, padding_zeros, targets, img_paths, indices=None):
    counter_inst = np.ones((num_categories), np.int)
    if indices == None:
        indices = xrange(len(targets))
        
    for j in indices:
        category_ind = targets[j]
        img_dir = os.path.join(split_dir, str(category_ind))
        
        num_img = counter_inst[category_ind]
        dst_basename = str(num_img).zfill(padding_zeros)
        dst_basename += ext(img_paths[j])
        dst_filepath = os.path.join(img_dir, dst_basename)

        os.symlink(img_paths[j], dst_filepath)
        counter_inst[category_ind] += 1

In [26]:
num_categories = len(categories)
num_images = sum([u.num_images for u in sorted_users])
padding_zeros = num_digits(num_images)

splits = ['training','test','validation']
filepaths = ['training_split.txt','validation_split.txt', 'test_split.txt']
for split, filepath in zip(splits, filepaths):
    
    split_dir = osp.join('data', split)
    if os.path.isdir(split_dir):
        shutil.rmtree(split_dir)    
    
    for j in xrange(num_categories): 
        category = str(j)
        category_dir = os.path.join(split_dir, category)
        makedirs(category_dir)
        
    targets = list()
    img_paths = list()
    for user_id, date in ntcir.read_split(filepath):
        for image in days[user_id][date].images:
            targets.append(image.label)
            img_paths.append(image.path)
    
    link_images(num_categories, split_dir, padding_zeros, targets, img_paths)